In [45]:
#setup_breakhis.py and run_mi_classify.py file in jupyter notebook
#made this change to understand the putput files required to run the program 

In [44]:
import sys
import os
from glob import glob
import numpy as np
import sklearn.model_selection
import sklearn.metrics

In [46]:
import util
from linear_classifier import LinearClassifier
from sil import SIL

In [5]:
def create_folds_file(file_name = 'folds1.txt', fold = '1'):

    column_names = ''
    file_handle = open(file_name,"w+")

    with open('BreaKHis_v1/Folds.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            fold_row = row[0]
            column_names = ",".join(row)
            if fold_row == 'fold' or fold_row == fold:
                file_handle.write(column_names)
                file_handle.write("\n")

    file_handle.close()

In [6]:
create_folds_file(file_name = 'folds1.txt', fold = '1')
create_folds_file(file_name = 'folds2.txt', fold = '2')
create_folds_file(file_name = 'folds3.txt', fold = '3')
create_folds_file(file_name = 'folds4.txt', fold = '4')
create_folds_file(file_name = 'folds5.txt', fold = '5')

In [40]:
#!python setup_breakhis.py -i BreaKHis_v1/ -o BreaKHis200/ --mag 200

root_dir = './BreaKHis_v1/histology_slides/breast'
srcfiles = {'DC': 'malignant/SOB/ductal_carcinoma/%s/%sX/%s',
                'LC': 'malignant/SOB/lobular_carcinoma/%s/%sX/%s',
                'MC': 'malignant/SOB/mucinous_carcinoma/%s/%sX/%s',
                'PC': 'malignant/SOB/papillary_carcinoma/%s/%sX/%s',
                'A': 'benign/SOB/adenosis/%s/%sX/%s',
                'F': 'benign/SOB/fibroadenoma/%s/%sX/%s',
                'PT': 'benign/SOB/phyllodes_tumor/%s/%sX/%s',
                'TA': 'benign/SOB/tubular_adenoma/%s/%sX/%s'}
sub_dir = 'histology_slides/breast/'
src_dir = os.path.join(os.getcwd(), 'BreaKHis_v1/')
print(os.path.exists(src_dir))
out_dir = os.path.join(os.getcwd(), 'BreaKHis200/')
if not os.path.exists(out_dir):
    print('Creating '+out_dir)
    os.makedirs(out_dir)
magnification = '200'

True


In [36]:
for nfold in range(5):
    
    db = open(src_dir+'dsfold%s.txt' % (nfold+1))
    print( 'Creating ' + out_dir + 'fold%s.txt' % nfold )
    fd_fold = open( out_dir + 'fold%s.txt' % nfold, 'w' )
    line_count = 0
    
    for row in db.readlines():
        columns = row.split(',')
        if line_count == 0:
            line_count += 1
        else:
            imgname = columns[3]
            mag = columns[1]  # 40, 100, 200, or 400
            grp = columns[2].strip()  # train or test
            #print(grp)

            s = imgname.split('-')
            name = s[0] + '-' + s[1] + '-' + s[2] + '-' + s[4].split('.')[0]
            #print(name)

            if mag != '200':
                continue

            if grp == 'train':
                #print(grp)
                fd_fold.write(name+',train\n')
            else:
                fd_fold.write(name+',test\n')

            #print(columns)
            #break
    
    fd_fold.close()
    
    db.close()

Creating /home/vibha/Documents/Research/project2/ImageMIL/BreaKHis200/fold0.txt
Creating /home/vibha/Documents/Research/project2/ImageMIL/BreaKHis200/fold1.txt
Creating /home/vibha/Documents/Research/project2/ImageMIL/BreaKHis200/fold2.txt
Creating /home/vibha/Documents/Research/project2/ImageMIL/BreaKHis200/fold3.txt
Creating /home/vibha/Documents/Research/project2/ImageMIL/BreaKHis200/fold4.txt


In [41]:
print('Creating '+out_dir + 'sample_images.csv')
fd_images = open( out_dir + 'sample_images.csv', 'w' )
    
for k,v in srcfiles.items():
    for fn in glob( src_dir + sub_dir+ v.replace('%sX',magnification+'X').replace('%s','*') + '.png' ):
        imgname = fn.split('/')[-1]
        s = imgname.split('-')
        name = s[0] + '-' + s[1] + '-' + s[2] + '-' + s[4].split('.')[0]
        tumor_type = imgname.split('-')[0].split('_')[-1]

        fn = fn[fn.find(src_dir)+len(src_dir):]
        fn = fn[fn.find(sub_dir)+len(sub_dir):]

        fd_images.write( name+','+fn+'\n' )

fd_images.close()

Creating /home/vibha/Documents/Research/project2/ImageMIL/BreaKHis200/sample_images.csv


In [42]:
print('Creating '+out_dir + 'labels.csv')
fd_class = open( out_dir + 'labels.csv', 'w' )
fd_class.write( 'sample,tumor,tumor_type,benign_type,malignant_type\n' )
    
for k,v in srcfiles.items():
    for fn in glob( src_dir + sub_dir+ v.replace('%sX',magnification+'X').replace('%s','*') + '.png' ):
        imgname = fn.split('/')[-1]
        s = imgname.split('-')
        name = s[0] + '-' + s[1] + '-' + s[2] + '-' + s[4].split('.')[0]
        tumor_type = imgname.split('-')[0].split('_')[-1]

        if 'malignant' in fn:
            tumor = 'M'
            benign_type = ''
            malignant_type = tumor_type
        else:
            tumor = 'B'
            benign_type = tumor_type
            malignant_type = ''
        fd_class.write( '%s,%s,%s,%s,%s\n' % ( name, tumor, tumor_type, benign_type, malignant_type ) )

fd_class.close()

Creating /home/vibha/Documents/Research/project2/ImageMIL/BreaKHis200/labels.csv
